In [2]:
# ============================================================
# CONSOLIDADOR FINAL - Bin Ensemble + Best Regression Model per Ticker
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# ============================================================
# CONFIGURAÇÕES
# ============================================================

BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/stock"
MODEL_DIR = f"{BASE_DIR}/models"
PARQUET_PATH = f"{BASE_DIR}/expanded_stock_reduced.parquet"

# Tags dos modelos
BIN_MODEL_TAG = "multi_model_timeaware_v3"
REG_MODEL_TAG = "v20_PER_TICKER_NORMALIZED"

# Arquivo de saída
OUTPUT_FILE = f"{BASE_DIR}/consolidated_final_best_models.csv"
LAST_DAYS = 5
MIN_QUALITY_SCORE = 0.7
ALLOW_FALLBACK = False

print("="*70)
print("CONSOLIDADOR FINAL - Melhor Modelo de Regressão por Ticker")
print("="*70)

# ============================================================
# 1. CARREGA MÉTRICAS DE REGRESSÃO E SELECIONA MELHOR MODELO
# ============================================================

print("\n[1/6] Selecionando melhor modelo de regressão por ticker...")

metrics_path = f"{MODEL_DIR}/{REG_MODEL_TAG}_outputs/metrics_valid.csv"
if not os.path.exists(metrics_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {metrics_path}")

metrics_df = pd.read_csv(metrics_path)

# Para cada ticker + target, seleciona modelo com menor RMSE
best_models = {}

for ticker in metrics_df['ticker'].unique():
    ticker_metrics = metrics_df[metrics_df['ticker'] == ticker]

    for target in ['target_best_entry', 'target_best_sale']:
        target_metrics = ticker_metrics[ticker_metrics['target'] == target]

        if target_metrics.empty:
            continue

        # Seleciona modelo com menor RMSE
        best_row = target_metrics.loc[target_metrics['RMSE'].idxmin()]

        key = (ticker, target)
        best_models[key] = {
            'model': best_row['model'],
            'MAE': best_row['MAE'],
            'RMSE': best_row['RMSE'],
            'R2': best_row['R2']
        }

print(f"  ✓ Melhor modelo selecionado para {len(best_models)} combinações ticker/target")

# Preview dos modelos selecionados
preview_selection = []
for (ticker, target), info in list(best_models.items())[:10]:
    preview_selection.append({
        'ticker': ticker,
        'target': target,
        'best_model': info['model'],
        'RMSE': info['RMSE'],
        'R2': info['R2']
    })

print("\n=== Preview: Melhores Modelos (primeiros 10) ===")
print(pd.DataFrame(preview_selection).to_string(index=False))

# ============================================================
# 2. CARREGA PREDIÇÕES DE REGRESSÃO
# ============================================================

print("\n[2/6] Carregando predições de regressão...")

reg_predictions_path = f"{MODEL_DIR}/{REG_MODEL_TAG}_outputs/apply_predictions_tidy.csv"
if not os.path.exists(reg_predictions_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {reg_predictions_path}")

reg_df = pd.read_csv(reg_predictions_path)
reg_df['Date'] = pd.to_datetime(reg_df['Date'])

# Filtra apenas os melhores modelos por ticker/target
best_models_df = pd.DataFrame(
    [
        {
            "ticker": ticker,
            "target": target,
            "model": info["model"],
            "RMSE": info["RMSE"],
            "R2": info["R2"],
        }
        for (ticker, target), info in best_models.items()
    ]
)

reg_df_best = reg_df.merge(
    best_models_df[["ticker", "target", "model"]],
    on=["ticker", "target", "model"],
    how="inner"
)

if 'quality_score' in reg_df_best.columns:
    reg_df_best = reg_df_best[reg_df_best['quality_score'] >= MIN_QUALITY_SCORE]

if not ALLOW_FALLBACK and 'used_fallback' in reg_df_best.columns:
    reg_df_best = reg_df_best[~reg_df_best['used_fallback']]

if reg_df_best.empty:
    raise ValueError(
        "Nenhuma predição disponível após filtros de qualidade. "
        "Ajuste MIN_QUALITY_SCORE/ALLOW_FALLBACK ou gere novos modelos."
    )

print(f"  ✓ Predições filtradas: {len(reg_df_best)} registros (apenas melhores modelos)")

# Limita aos últimos dias configurados
last_dates = sorted(reg_df_best['Date'].unique())
if LAST_DAYS is not None and len(last_dates) > LAST_DAYS:
    last_dates = last_dates[-LAST_DAYS:]
    reg_df_best = reg_df_best[reg_df_best['Date'].isin(last_dates)]

# Pega lista de tickers
tickers_all = sorted(reg_df_best['ticker'].unique())

# ============================================================
# 3. BAIXA PREÇOS REAIS DO YAHOO FINANCE
# ============================================================

print(f"\n[3/6] Baixando preços Close via yfinance...")

last_dates = sorted(reg_df_best['Date'].unique())
start_date = pd.to_datetime(last_dates[0]) - timedelta(days=30)
end_date = pd.to_datetime(last_dates[-1]) + timedelta(days=1)

prices_data = yf.download(
    tickers=tickers_all,
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'),
    group_by='column',
    auto_adjust=True,
    progress=False,
    threads=True
)

# Extrai Close
if 'Close' in prices_data.columns:
    close_prices = prices_data['Close'].copy()
elif len(tickers_all) == 1:
    close_prices = pd.DataFrame({tickers_all[0]: prices_data['Close']})
else:
    close_prices = prices_data.xs('Close', level=0, axis=1)

close_prices = close_prices.ffill().bfill()
close_prices = close_prices[close_prices.index.isin(pd.to_datetime(last_dates))]

# Converte para formato long
price_rows = []
for date in last_dates:
    date_pd = pd.to_datetime(date)
    if date_pd not in close_prices.index:
        continue
    for ticker in tickers_all:
        if ticker not in close_prices.columns:
            price = np.nan
        else:
            price = close_prices.loc[date_pd, ticker]

        price_rows.append({
            'Date': date_pd,
            'ticker': ticker,
            'Close': float(price) if pd.notna(price) else np.nan
        })

price_df = pd.DataFrame(price_rows)

print(f"  ✓ Preços válidos: {(~price_df['Close'].isna()).sum()}/{len(price_df)} ({(~price_df['Close'].isna()).sum()/len(price_df)*100:.1f}%)")

# ============================================================
# 4. CONVERTE PERCENTUAIS PARA PREÇOS ABSOLUTOS
# ============================================================

print("\n[4/6] Convertendo percentuais para preços absolutos...")

reg_with_close = reg_df_best.merge(price_df, on=['Date', 'ticker'], how='left')
reg_with_close['pred_price'] = reg_with_close['Close'] * (1 + reg_with_close['pred_pct'])

# ============================================================
# 5. CARREGA PREDIÇÕES DE ENSEMBLE BINÁRIO
# ============================================================

print("\n[5/6] Carregando predições de ensemble binário...")

# Procura arquivos do ensemble binário
bin_patterns = [
    f"{MODEL_DIR}/{BIN_MODEL_TAG}/apply_predictions.csv",
    f"{MODEL_DIR}/apply_predictions_bin.csv",
    f"{BASE_DIR}/apply_bin_ensemble.csv",
]

bin_df = None
for pattern in bin_patterns:
    if os.path.exists(pattern):
        bin_df = pd.read_csv(pattern)
        print(f"  ✓ Encontrado: {pattern}")
        break

if bin_df is None:
    raise FileNotFoundError(
        "Nenhum arquivo de predição binária encontrado. "
        "Verifique BIN_MODEL_TAG e o diretório de modelos antes de consolidar."
    )
else:
    if 'Date' in bin_df.columns:
        bin_df['Date'] = pd.to_datetime(bin_df['Date'])
    elif bin_df.index.name == 'Date' or pd.api.types.is_datetime64_any_dtype(bin_df.index):
        bin_df = bin_df.reset_index()
        bin_df['Date'] = pd.to_datetime(bin_df['Date'])

bin_df = bin_df[bin_df['Date'].isin(pd.to_datetime(last_dates))].copy()

print(f"  ✓ Predições binárias: {len(bin_df)} registros")

# ============================================================
# 6. CONSOLIDA TABELA FINAL
# ============================================================

print("\n[6/6] Consolidando tabela final...")

# Pivota regressão para ter entry e sale em colunas separadas
reg_pivot = reg_with_close.pivot_table(
    index=['Date', 'ticker'],
    columns='target',
    values='pred_price',
    aggfunc='first'
).reset_index()

reg_pivot.columns.name = None

if 'target_best_entry' in reg_pivot.columns:
    reg_pivot.rename(columns={'target_best_entry': 'pred_best_entry'}, inplace=True)
if 'target_best_sale' in reg_pivot.columns:
    reg_pivot.rename(columns={'target_best_sale': 'pred_best_sale'}, inplace=True)

# Adiciona métricas do melhor modelo por ticker
for _, row in reg_pivot.iterrows():
    ticker = row['ticker']

    # Métricas para entry
    key_entry = (ticker, 'target_best_entry')
    if key_entry in best_models:
        reg_pivot.loc[
            (reg_pivot['ticker'] == ticker) & (reg_pivot['Date'] == row['Date']),
            'R2_entry'
        ] = best_models[key_entry]['R2']
        reg_pivot.loc[
            (reg_pivot['ticker'] == ticker) & (reg_pivot['Date'] == row['Date']),
            'RMSE_entry'
        ] = best_models[key_entry]['RMSE']

    # Métricas para sale
    key_sale = (ticker, 'target_best_sale')
    if key_sale in best_models:
        reg_pivot.loc[
            (reg_pivot['ticker'] == ticker) & (reg_pivot['Date'] == row['Date']),
            'R2_sale'
        ] = best_models[key_sale]['R2']
        reg_pivot.loc[
            (reg_pivot['ticker'] == ticker) & (reg_pivot['Date'] == row['Date']),
            'RMSE_sale'
        ] = best_models[key_sale]['RMSE']

# Merge: price + bin + reg
final_df = price_df.merge(bin_df, on=['Date', 'ticker'], how='left')
final_df = final_df.merge(reg_pivot, on=['Date', 'ticker'], how='left')

# Reordena colunas conforme especificação
col_order = [
    'Date',
    'ticker',
    'Close',
    'proba_target_up20__ens_weighted',
    'pred_target_up20__ens_weighted',
    'proba_target_dd5__ens_weighted',
    'pred_target_dd5__ens_weighted',
    'AP_metric_up20',
    'AP_metric_dd5',
    'pred_best_entry',
    'pred_best_sale',
    'R2_entry',
    'RMSE_entry',
    'R2_sale',
    'RMSE_sale',
]

col_order = [c for c in col_order if c in final_df.columns]
final_df = final_df[col_order].copy()
final_df = final_df.sort_values(['Date', 'ticker']).reset_index(drop=True)

print(f"  ✓ Tabela final: {final_df.shape[0]} linhas × {final_df.shape[1]} colunas")

# ============================================================
# 7. SALVA ARQUIVO
# ============================================================

print(f"\n[SALVANDO] {OUTPUT_FILE}")
final_df.to_csv(OUTPUT_FILE, index=False, float_format='%.6f')
print("  ✓ Arquivo salvo!")

# Salva também mapeamento de melhores modelos
best_models_export = []
for (ticker, target), info in best_models.items():
    best_models_export.append({
        'ticker': ticker,
        'target': target,
        'best_model': info['model'],
        'MAE': info['MAE'],
        'RMSE': info['RMSE'],
        'R2': info['R2']
    })

best_models_df = pd.DataFrame(best_models_export)
best_models_path = OUTPUT_FILE.replace('.csv', '_best_models_map.csv')
best_models_df.to_csv(best_models_path, index=False, float_format='%.6f')
print(f"  ✓ {best_models_path}")

# ============================================================
# 8. PREVIEW E ESTATÍSTICAS
# ============================================================

print("\n" + "="*70)
print("PREVIEW DA TABELA CONSOLIDADA (primeiras 30 linhas)")
print("="*70)
print(final_df.head(30).to_string(index=False))

print("\n" + "="*70)
print("ESTATÍSTICAS GERAIS")
print("="*70)
print(f"📊 Total de registros: {len(final_df)}")
print(f"📅 Datas: {final_df['Date'].min().date()} → {final_df['Date'].max().date()}")
print(f"🎯 Tickers: {final_df['ticker'].nunique()}")
print(f"💰 Preço médio (Close): ${final_df['Close'].mean():.2f}")

# Estatísticas dos modelos selecionados
model_counts = best_models_df['best_model'].value_counts()
print(f"\n📈 Distribuição de modelos selecionados:")
for model, count in model_counts.items():
    print(f"   {model}: {count} ({count/len(best_models_df)*100:.1f}%)")

# Estatísticas de R²
print(f"\n🎯 R² médio (entry): {final_df['R2_entry'].mean():.4f}")
print(f"🎯 R² médio (sale): {final_df['R2_sale'].mean():.4f}")
print(f"📉 RMSE médio (entry): {final_df['RMSE_entry'].mean():.6f}")
print(f"📉 RMSE médio (sale): {final_df['RMSE_sale'].mean():.6f}")

# Top tickers por R² (entry)
print(f"\n🏆 Top 10 Tickers por R² (entry):")
top_r2 = final_df.groupby('ticker')['R2_entry'].mean().sort_values(ascending=False).head(10)
for ticker, r2 in top_r2.items():
    print(f"   {ticker}: {r2:.4f}")

print("\n✅ Consolidação concluída com sucesso!")
print(f"📁 Arquivos gerados:")
print(f"   1. {OUTPUT_FILE}")
print(f"   2. {best_models_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CONSOLIDADOR FINAL - Melhor Modelo de Regressão por Ticker

[1/6] Selecionando melhor modelo de regressão por ticker...
  ✓ Melhor modelo selecionado para 294 combinações ticker/target

=== Preview: Melhores Modelos (primeiros 10) ===
   ticker            target best_model     RMSE        R2
 ABEV3.SA target_best_entry       lgbm 0.029836 -0.086655
 ABEV3.SA  target_best_sale       lgbm 0.049337 -1.721439
ALZR11.SA target_best_entry        xgb 0.020547 -1.785056
ALZR11.SA  target_best_sale       lgbm 0.017727 -0.241701
 AUDUSD=X target_best_entry       lgbm 0.009096  0.305858
 AUDUSD=X  target_best_sale        xgb 0.043268 -1.387617
 AZUL4.SA target_best_entry        xgb 0.127366 -5.319257
 AZUL4.SA  target_best_sale       lgbm 0.070739 -0.571286
 B3SA3.SA target_best_entry       lgbm 0.025797  0.621707
 B3SA3.SA  target_best_sale        xgb 0.054393 -0.13072

ERROR:yfinance:
14 Failed downloads:
ERROR:yfinance:['XAGEUR', 'CESP6.SA', 'GOLL4.SA', 'JBSS3.SA', 'LAME4.SA', 'XAUEUR', 'CCRO3.SA', 'CIEL3.SA', 'NTCO3.SA', 'VVAR3.SA']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:['S=F', 'BRFS3.SA', '^SSE', 'FIBR3.SA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-09-29 -> 2025-11-03)')


  ✓ Preços válidos: 665/735 (90.5%)

[4/6] Convertendo percentuais para preços absolutos...

[5/6] Carregando predições de ensemble binário...
  ⚠️ Simulando dados do modelo binário...
  ✓ Predições binárias: 735 registros

[6/6] Consolidando tabela final...
  ✓ Tabela final: 735 linhas × 15 colunas

[SALVANDO] /content/drive/MyDrive/Colab Notebooks/stock/consolidated_final_best_models.csv
  ✓ Arquivo salvo!
  ✓ /content/drive/MyDrive/Colab Notebooks/stock/consolidated_final_best_models_best_models_map.csv

PREVIEW DA TABELA CONSOLIDADA (primeiras 30 linhas)
      Date    ticker         Close  proba_target_up20__ens_weighted  pred_target_up20__ens_weighted  proba_target_dd5__ens_weighted  pred_target_dd5__ens_weighted  AP_metric_up20  AP_metric_dd5  pred_best_entry  pred_best_sale   R2_entry   RMSE_entry    R2_sale    RMSE_sale
2025-10-29  ABEV3.SA     12.030000                         0.499063                               0                        0.272561                             